# Add Assignments From an Existing Layer
In this example, features in an existing layer on ArcGIS Online, representing crosswalks that need to be painted, will be imported into a Workforce Project as assignments.

In [ ]:
from datetime import datetime
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce

### Connect to Organization and get the Project
Let's connect to ArcGIS Online and find the new Project to add assignments to.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("1f7b42024da544f6b1e557889e858ac6")
project = workforce.Project(item)

### Get the "Paint Crosswalk" Assignment Type and the Dispatcher to Use
Let's get the "Paint Crosswalk" assignment type and the dispatcher from the Project.

In [ ]:
paint_crosswalk = project.assignment_types.get(name="Paint Crosswalk")
dispatcher = project.dispatchers.get(user_id="workforce_scripts")

### Query the Feature Layer to Find Crosswalks That Need to be Painted
Let's fetch the layer containing the crosswalks, query all of the crosswalks, and then display the crosswalks on a map. Workforce stores assignments in WGS84 Web Mercator so let's ensure that the returned geometries are using that spatial reference.

In [ ]:
layer = gis.content.get("2ec0051ba3834714a639365dace5c4c0").layers[0]
crosswalks = layer.query("1=1", out_sr=3857).features

In [ ]:
crosswalk_map = gis.map("San Diego, CA", zoomlevel=12)
crosswalk_map.add_layer(layer)
crosswalk_map

### Create an Assignment for Each Feature
For each assignment, let's use the geometry and attributes of the existing feature to create the assignment. Then let's use the batch_add method to add multiple assignments at once (this is faster than using the add method since validation is performed once for all assignments).

In [ ]:
assignments = []
for feature in crosswalks:
    assignments.append(
        workforce.Assignment(
            project,
            geometry=feature.geometry,
            location=feature.attributes["location"],
            description=feature.attributes["description"],
            priority=feature.attributes["priority"],
            assignment_type=paint_crosswalk,
            dispatcher=dispatcher,
            status="unassigned"
        )
    )
project.assignments.batch_add(assignments)

### Verify Assignments on the Map

In [ ]:
webmap = gis.map("San Diego", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap